# Step 0: Import FLDPLN Library and Establish Folder Path

In [ ]:
import sys
import time

import pathlib
from shapely.geometry import Polygon

import os

# import the mapping module in the fldpln package
# import DASK libraries for parallel mapping
from dask.distributed import Client, LocalCluster
from dask import visualize

# import the mapping and gauge modules from the fldpln package
from fldpln.mapping import *
from fldpln.gauge import *

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, Resampling
from collections import defaultdict
from itertools import product
from collections import defaultdict
from shapely.geometry import Point
from rasterio.transform import from_origin
from scipy.signal import savgol_filter
from scipy.stats import rankdata


### Set Mapping Parameters
Select the tiled library folder for study area and select the library to be mapped. 

Setup the map folder (i.e., outMapFolderName) which is under the output folder and comtains all inundation depth maps. Additional settings include whether to mosaic tiles as single COG file and whether use a Dask local cluster to speed up the mapping.

In [ ]:
# Establish working directory. All notebooks, FLDPLN python library, and study area terrain data should be in the directory
work_dir = pathlib.Path().resolve()
print('Working directory: ', work_dir)

# tiled library folder
libFolder =  'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/tiled_snz_library'
print('Tile library: ', libFolder)

# libraries to be mapped
allLibNames = ['lib_fldsensing']

#Filled DEM location

fil_path = str(work_dir / r'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/bil/fil.bil')


# Set output folder for location of all map outputs
outputFolder = 'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/maps'

# set up map folder
outMapFolderName = 'testingSpecial5'
# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)
print('Output maps stored in: ', outputFolder)

if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

# whether mosaci tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False # This doesn't work on my office desktop though it works fine on KBS server
numOfWorkers = round(0.8*os.cpu_count())
numOfWorkers = 6
print(f'Number of workers: {numOfWorkers}')

## Note
In FLDPLN code, the procedure first snap and interpolate stage between FSPs and later, given the value, map the corresponding FPPs. For the method proposed, technically this interpolation step should be performed after the RS edge selection and later, deriving the stage that generates the DoF close to zero at the FPP.

*Steps*

1. Load "clean edge" raster
2. Snap raster to .bil or similar
3. Extract X,Y coordinates of edge pixels into a df
4. Open FLDPLN_tiled_tile_index.csv to extract info of .snz tiles and edge pixel locations.

# Step 1: Reproject true edge tif file into the same projection and alignment with FLDPLN 

In [ ]:
def snap_raster(src_path, ref_path, dst_path):
    '''
    This function reprojects and alligns the true edge tiff file into the same projection with FLDPLN library and save the new projected
    file.
    
    args:
        src_path (str): Directory of true edge tif file 
        ref_path (str): Directory of FLDPLN DEM bil file
        dst_path (str): Directory of new reprojected true edge file should be 
    '''
    with rasterio.open(ref_path) as ref_src:
        ref_transform = ref_src.transform
        ref_crs = ref_src.crs
        ref_width = ref_src.width
        ref_height = ref_src.height

    with rasterio.open(src_path) as src:
        profile = src.profile
        profile.update({
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        with rasterio.open(dst_path, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=ref_transform,
                    dst_crs=ref_crs,
                    resampling=Resampling.nearest
                )
    print('New true edge raster saved at: %s'%(dst_path))
    

### Note: If the reprojected true edge raster has not been made, uncomment the cell below and run

In [ ]:
src_path = str(work_dir / r'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/RS/hecrasedge.tif')
dst_path = str(work_dir / r'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/RS/hecrasedge_align.tif')    
ref_path = str(work_dir / r'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/bil/dem.bil')
snap_raster(src_path, ref_path, dst_path)

### Else, run the cell below to establish alligned, reprojected true edge raster path

In [ ]:
dst_path = 'C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/RS/hecrasedge_align.tif' 
print(dst_path)

# Step 2: Build DataFrame of true edge pixels and their associated index in FLDPLN library

In [ ]:
def extract_coordinates(raster_path, value):
    '''
    This function transforms true edge raster into dataframe of X/Y coordinates.
    
    args:
        raster_path (str): Directory of true edge raster (should be reprojected and aligned)
        value (int/float): Value that represents true edge (usually 1)

    return:
        pd.DataFrame
    '''
    
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        transform = src.transform

    rows, cols = np.where(data == value)
    xs, ys = rasterio.transform.xy(transform, rows, cols)

    df = pd.DataFrame({
        'X_coord': xs,
        'Y_coord': ys
    })
    return df

## Function to find the tile ID for a given FPP coordinate
def find_tile_id_and_indices(x, y, tile_index_df, cell_size):
    '''
    This function searchs the tile ID and row/col value of a given FPP X/Y coordinate based on the information 
    of Max/Min X/Y of each tile.  
    
    args:
        x (int/float): X coordinate (Lon)
        y (int/float): Y coordinate (Lat)
        tile_index_df (pd.DataFrame): tile information dataframe
        cell_size (int/float): spatial resolution in meter

    return:
        int, int, int: tile ID, FPP col index, FPP row index
    '''
    for idx, row in tile_index_df.iterrows():
        if row['TileMinX'] < x < row['TileMaxX'] and row['TileMinY'] < y < row['TileMaxY']:
            fpp_col = int((x - (row['FppMinX'] + hcs)) / cell_size)
            fpp_row = int(((row['FppMaxY'] - hcs) - y) / cell_size)
            return row['TileId'], fpp_col, fpp_row
    return None, None, None

def extract_elevation_from_dem(filledDem_path, coordinates_df):
    '''
    Extract elevation values from a DEM for a given set of coordinates.

    args:
        dem_path (str): Path to the DEM file.
        coordinates_df (pd.DataFrame): DataFrame containing X/Y coordinates (columns: 'X_coord', 'Y_coord').

    return:
        Input DataFrame with an additional 'FPP_Elevation' column.
    '''
    with rasterio.open(filledDem_path) as dem:
        # Prepare coordinates as a list of tuples
        coords = list(zip(coordinates_df['X_coord'], coordinates_df['Y_coord']))

        # Sample elevation values from the DEM
        elevation_values = [
            val[0] if val else np.nan  # Handle None values returned by rasterio.sample
            for val in dem.sample(coords)
        ]

        # Add the elevation values to the DataFrame
        coordinates_df['FPP_Elevation'] = elevation_values

    return coordinates_df

# Create dataframe of True edge coordinate
value_to_extract = 1
RS_edges = extract_coordinates(dst_path, value_to_extract)

## Load the tile index CSV file
tile_index_path = str(pathlib.Path(libFolder) / allLibNames[0] / 'FLDPLN_tiled_tile_index.csv')
tile_index_df = pd.read_csv(tile_index_path)
cell_size = 10
hcs = cell_size / 2
tile_index_df.head()

## Apply the function to each row in the coordinates DataFrame to obtain tile ID and row/col
RS_edges[['TileId', 'FppCol', 'FppRow']] = RS_edges.apply(
    lambda row: pd.Series(find_tile_id_and_indices(row['X_coord'], row['Y_coord'], tile_index_df,cell_size)), axis=1)
RS_edges_with_elevation = extract_elevation_from_dem(fil_path, RS_edges)
## Display the first few rows of the DataFrame with Tile IDs and FppCols/FppRows
RS_edges

# Step 3: Read tile .snz file and query DTFs relationship of true edge FPP

In [ ]:
## Function to get the FspIds and Dtfs for a specific coordinate from the loaded tile data
def get_fspids_and_dtfs_for_coordinate(fpp_col, fpp_row, tile_id, tile_data):
    '''
    This function saves all FspIDs, Dtfs , and the FilledDepth values that correspond to the FPP   
    
    args:
        fpp_col (int): Column index of the FPP in the tile
        fpp_row (int): Row index of the FPP in the tile
        tile_id (int): The id of the tile
        tile_data (dict): Dictionary that contain all tile data from the .snz files 

    return:
        list, list, list: FspID list, Dtf list, FilledDepth list
    '''
    
    tile_df = tile_data.get(tile_id)
    if tile_df is not None:
        results = tile_df[(tile_df['FppCol'] == fpp_col) & (tile_df['FppRow'] == fpp_row)]
        if not results.empty:
            return results['FspId'].tolist(), results['Dtf'].tolist(), results['FilledDepth'].tolist()
    return [], [], []

## Get unique tile IDs needed for the coordinates
dummy = RS_edges['TileId'].dropna().unique()
unique_tile_ids = dummy.astype(int)
print('Tiles to load:'+str(unique_tile_ids))

# Load data for each required tile

tile_data = {}
for tile_id in unique_tile_ids:
    snz_path = str(pathlib.Path(libFolder) / allLibNames[0] / str('FLDPLN_tiled_%s.snz'%(tile_id)))
    tile_data[tile_id] = pd.read_parquet(snz_path)

## Apply the data extraction function to each row in the coordinates DataFrame
RS_edges[['FspIds', 'Dtfs', 'FilledDepth']] = RS_edges.apply(
    lambda row: pd.Series(get_fspids_and_dtfs_for_coordinate(row['FppCol'], row['FppRow'], row['TileId'], tile_data)), axis=1)

RS_edges.head()

## Filter the Edge Dataframe to remove edges with no FSP relationship. Add a column that count number of FSPs for each FPP
RS_edges_filtered = RS_edges.copy()
RS_edges_filtered['NumsFSP'] = RS_edges['FspIds'].map(len)
RS_edges_filtered = RS_edges_filtered.drop(RS_edges_filtered[RS_edges_filtered['NumsFSP'] == 0].index).reset_index(drop = True)
RS_edges_filtered

# Step 4: Create GeoDataframes of the true edge dataframe and of the FSPs

In [ ]:
# Map a given Fpp and its Fsp by creating GeoDataframe
gdfRS_edges_filtered = gpd.GeoDataFrame(RS_edges_filtered, geometry=gpd.points_from_xy(RS_edges_filtered['X_coord'], RS_edges_filtered['Y_coord']))
gdfRS_edges = gpd.GeoDataFrame(RS_edges, geometry=gpd.points_from_xy(RS_edges['X_coord'], RS_edges['Y_coord']))

# Set the coordinate reference system (CRS) to EPSG 26914 for NAD UTM 14N. Change if required
# It is possible to load .prj from folder for this procedure but we set it here for convenience

gdfRS_edges_filtered = gdfRS_edges_filtered.set_crs(epsg=26914)

# Save the maximum depth/DTF of each FPP 
max_depth = gdfRS_edges_filtered.Dtfs.apply(lambda x:np.max(x))
gdfRS_edges_filtered['max_depth'] = max_depth
gdfRS_edges_filtered

## Note: To reduce outliers you must remove pixels that have an unrealist Depth To Flood value. To do this an average max depth can be taken and whatever pixel is above the average will be filtered. 

In [ ]:
average = sum(max_depth)/len(max_depth)

print("Pixels above following number will be masked out: ", average)
max_dtf_threshold = average

gdfRS_edges_filtered = gdfRS_edges_filtered[gdfRS_edges_filtered['max_depth'] < max_dtf_threshold].reset_index(drop = True)

In [ ]:
# Create the FSP info GeoDataframe
fsp_path = str(pathlib.Path(libFolder) / allLibNames[0] / 'fsp_info.csv')
print(libFolder)
fsp_info = pd.read_csv(fsp_path)

print(fsp_path)
fsp_gdf = gpd.GeoDataFrame(fsp_info, geometry=gpd.points_from_xy(fsp_info['FspX'], fsp_info['FspY']))
## Set the coordinate reference system (CRS) to EPSG 26914
fsp_gdf = fsp_gdf.set_crs(epsg=26914, inplace=True)
fspGDFPlot=fsp_gdf

# Step 5: Plot Geodataframe

In [ ]:
# PLOT FSP river profile and FPP at true edge pixel from RS

## Normalize the FspId_count for color mapping
# norm = Normalize(vmin=gdfRS_edges_filtered['NumsFSP'].min(), vmax=gdfRS_edges_filtered['NumsFSP'].max())
norm = Normalize(vmin=gdfRS_edges_filtered['max_depth'].min(), vmax=gdfRS_edges_filtered['max_depth'].max())
cmap = plt.cm.viridis

## Plot the data
fig, ax = plt.subplots(1, 1, figsize=(4, 6))

# gdfRS_edges_filtered.plot(column='NumsFSP', cmap=cmap, markersize=100, ax=ax, legend=False, norm=norm)
gdfRS_edges_filtered.plot(column='max_depth', cmap=cmap, markersize=100, ax=ax, legend=False, norm=norm)

fsp_gdf[ (fsp_gdf.StrOrd == 1)|(fsp_gdf.StrOrd == 2) | (fsp_gdf.StrOrd == 3)|(fsp_gdf.StrOrd == 4) | (fsp_gdf.StrOrd == 5)|(fsp_gdf.StrOrd == 6) | (fsp_gdf.StrOrd == 7)| (fsp_gdf.StrOrd == 8)].plot(ax=ax, color='green', markersize=2, label='Fsp')

for i in unique_tile_ids:
    row = tile_index_df.loc[tile_index_df['TileId'] == i, ['TileMinX', 'TileMaxX', 'TileMinY', 'TileMaxY']]
    if not row.empty:
        minx,maxx,miny,maxy = row.iloc[0]

    rectangle = Polygon([(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy), (minx, miny)])

    # Create a GeoDataFrame
    rec = gpd.GeoDataFrame([1], geometry=[rectangle], crs="EPSG:26914")
    rec.plot(ax=ax, edgecolor='black', markersize=1, facecolor='none')

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Max DTF')

ax.set_title('Fpps with Max DTF')
ax.set_xlabel('X Coordinate (EPSG 26914)')
ax.set_ylabel('Y Coordinate (EPSG 26914)')

plt.show()
gdfRS_edges_filtered.head()


In [ ]:
#Step 5.1 - Map a given Fpp and all Fsp related to it
## Function to plot the user-provided point and related Fsp points

def plot_user_point_and_fsps(user_point_index,gdf,fsp_gdf):
    user_point = gdf.iloc[user_point_index]
    related_fsp_ids = user_point['FspIds']
    related_fsps = fsp_gdf[fsp_gdf['FspId'].isin(related_fsp_ids)]
    
    stream_order_colors = {
        1: 'darkgreen',
        2: 'blue',
        3: 'orange',
        4: 'purple',
        5: 'darkred',
        6: 'pink',
        7: 'gray',
        8: 'black',
        9:'royalblue',
        10:'brown',
        11:'yellow',
        12:'darkviolet',
        13:'coral',
        14:'aqua',
        15:'dodgerblue',
        16:'grey',
        17:'salmon',
        18:'olive',
        19:'turquoise',
        20:'gold',
    }
    # Plot the data
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    for str_ord, color in stream_order_colors.items():
        # Filter Fsp points based on stream order and plot
        fsp_subset = fsp_gdf[fsp_gdf['StrOrd'] == str_ord]
        fsp_subset.plot(ax=ax, color=color, markersize=2, label=f'Stream Order {str_ord}(Color: {color})')
    # Plot the user-provided point
    gdf.plot(ax=ax, color='cyan', markersize=5, label='RS Fpp(Color: Cyan)')
    
    user_point_gdf = gpd.GeoDataFrame([user_point], geometry=[user_point.geometry])
    user_point_gdf.plot(ax=ax, color='red', markersize=50, label=f'User Provided Point (Color: red)')
    
    # Plot the related Fsp points
    related_fsps.plot(ax=ax, color='lawngreen', markersize=25, label='Related Fsp (Color: Lime Green)')
    
    # Set plot title and labels
    ax.set_title('Edge Pixels and Related Fsp Points')
    ax.set_xlabel('X Coordinate (EPSG 26914)')
    ax.set_ylabel('Y Coordinate (EPSG 26914)')
    
    plt.legend()
    plt.show()
print(len(gdfRS_edges_filtered))
plot_user_point_and_fsps(0,gdfRS_edges_filtered,fspGDFPlot) #Change the number of the pixel here, default is 1
fspGDFPlot

# Step 6: Assign Dof to FSP from RS true edge 

## Note: In our study, we filtered out FPP at edges that have DTFs relationship higher than 10. Customize the threshold n in the cell below

In [ ]:
# Add a threshold of NumofFSPs here for a subset of edges relationship. Can be used if number of FSPS causes program slowdown
#n = 10
gdfRS_edges_match_full = gdfRS_edges_filtered.copy()

# Run this instead to have no limitation of NumofFSP
n = gdfRS_edges_match_full.NumsFSP.max() 

gdfRS_edges_match = gdfRS_edges_match_full[gdfRS_edges_match_full.NumsFSP <= n].reset_index(drop = True) # Limit to edge with <= n Fsps
gdfRS_edges_match.Dtfs = gdfRS_edges_match.Dtfs.apply(lambda x:np.round(x,1)) # Round the dtfs by 1 decimal to reduce possible 
gdfRS_edges_match.head()


In [ ]:
# Create a dictionary mapping FspId to its coordinates from FspGDF
fsp_coord_dict = fsp_gdf[['FspId', 'FspX', 'FspY', 'FilledElev']].set_index('FspId').to_dict('index')
#print(fsp_coord_dict)
# Function to get coordinates for a list of FspIds
def get_fsp_coordinates(fsp_ids):
    fsp_coords = []
    for fsp_id in fsp_ids:
        if fsp_id in fsp_coord_dict:
            coords = fsp_coord_dict[fsp_id]
            fsp_coords.append((coords['FspX'], coords['FspY']))
    return fsp_coords
def get_fsp_filledElevations(fsp_ids):
    fsp_elevations = []
    for fsp_id in fsp_ids:
        if fsp_id in fsp_coord_dict:
            elevation = fsp_coord_dict[fsp_id]
            fsp_elevations.append((elevation['FilledElev']))
    return fsp_elevations
    
# Add new columns for FSP coordinates
gdfRS_edges_match_full['FSP_Coordinates'] = gdfRS_edges_match_full['FspIds'].apply(get_fsp_coordinates)

# Add new coloumn for FSP elevation
gdfRS_edges_match_full['FSP_Elevation'] = gdfRS_edges_match_full['FspIds'].apply(get_fsp_filledElevations)

# Split the coordinates into separate X and Y lists
gdfRS_edges_match_full['FSP_X_Coords'] = gdfRS_edges_match_full['FSP_Coordinates'].apply(lambda x: [coord[0] for coord in x])
gdfRS_edges_match_full['FSP_Y_Coords'] = gdfRS_edges_match_full['FSP_Coordinates'].apply(lambda x: [coord[1] for coord in x])

gdfRS_edges_match_full = gdfRS_edges_match_full.drop('FSP_Coordinates', axis=1, errors='ignore')

# Display the first few rows of the updated dataframe
gdfRS_edges_match_full.head()

# Step 6
### Note: There are two ways to assign Dof of FSP. One way is to use the median of all possible DTFs options for each FSP based on their relationships to FPP flood edge. Another way is to going through all possible combination of FSP Dof assignment and find the combination that provides the lowest flood depth across all flood edge pixels. 


In [ ]:
def get_depth(fsp,dtf,dof):
    '''
    This function calculates the flood depth at a certain FPP given the Flood Source Pixel (FSP) that can flood 
    it, the Depth To Flood (DTF) relationship, and the Depth of Flood (DOF) or stage of the FSP. The flood depth
    is calculated as the maximum of (DOF - DTF) for each FSP. 
    
    If flood depth is less than 0 (no FSP can flood the FPP), return nan
    
    args:
        fsp (np.arary/list): FSP list
        dtf (np.arary/list): corresponding DTF of the FSP
        dof (np.array/list): corresponding DOF/stage of the FSP
        
    return:
        float: flood depth value
        nan: if FPP is not flooded
    '''
    fsp, dtf, dof = np.array(fsp), np.array(dtf), np.array(dof)
    
    depth = np.max(dof - dtf)
    #print(f"{depth:.2f} = {dof} - {dtf}")
    
    if depth >= 0:
        return depth
    else:
        return np.nan


def filter_close_values(input_dict, threshold):
    '''
    This function reduces the number of options for each FSPs to reduce computational time. Given a certain threshold, 
    no two values within each list are closer than the threshold.
    
    args:
        input_dict (dict): Dictionary of key:value being FspID (int): Dof options (list)
        threshold (float): Threshold to remove values close to each other
    
    return:
        dict: filtered out FSP Dof functions 
    '''
    
    filtered_dict = {}
    for key, values in input_dict.items():
        # Sort values in descending order
        sorted_values = sorted(values, reverse=True)
        filtered_values = []
        for value in sorted_values:
            if all(abs(value - fv) > threshold for fv in filtered_values):
                filtered_values.append(value)
        
        filtered_dict[key] = filtered_values
    
    return filtered_dict


def num_of_combination(possible_fsp_dtf):
    '''
    This function calculate the number of possible combinations for FSP assignments 
    
    args:
        possible_fsp_dtf (dict): Dictionary of key:value being FspID (int): Dof options (list)
        
    return:
        int: number of combinations
    '''
    a = 1
    for key, value in possible_fsp_dtf.items():
        a = a*len(value)
    return a

In [ ]:
# 1. Setting Maximum Combinations and Initializing `best_case`
max_combinations = 20000  # Maximum combinations to avoid excessive computation
best_case = {}  # Store the best-case DOF for each FSP
all_combinations_df = pd.DataFrame(columns=['Combination_Index', 'Edge_Index', 'FSP_ID', 'Flood_Depth', 'Distance', 'Combined_Rank'])
final_results_df = pd.DataFrame(columns=['Edge_Index', 'Best_FSP', 'Dof', 'Flood_Depth', 'Distance', 'Combined_Rank'])
final_records = []
fsp_assignment_count = defaultdict(int)


# 2. Iterate through Different Limits of FSP Count per Edge
for i in range(gdfRS_edges_match_full.NumsFSP.min(), n + 1):
    print(f"Num of FSPs <= {i}")
    
    # 3. Filter Data by FSP Count and Round `Dtfs` Values
    gdfRS_edges_match = gdfRS_edges_match_full[gdfRS_edges_match_full.NumsFSP <= i].reset_index(drop=True)
    gdfRS_edges_match.Dtfs = gdfRS_edges_match.Dtfs.apply(lambda x: np.round(x, 1))  # Reduce unique combinations
    
    # 4. Generate Possible DTF Values for Each FSP
    possible_fsp_dtf = defaultdict(set)
    for fspids, dtfs in zip(gdfRS_edges_match['FspIds'], gdfRS_edges_match['Dtfs']):
        for fspid, dtf in zip(fspids, dtfs):
            possible_fsp_dtf[fspid].add(dtf)

    possible_fsp_dtf = {k: list(v) for k, v in possible_fsp_dtf.items()}
    
    for key, value in best_case.items():
        possible_fsp_dtf[key] = [value]

    # Calculate initial number of combinations
    num_choice = num_of_combination(possible_fsp_dtf)
    print(f"There are {num_choice} options of FSP DOF")

    # Reduce options if combinations exceed the threshold
    threshold = 0.1
    while num_choice > max_combinations:
        possible_fsp_dtf = filter_close_values(possible_fsp_dtf, threshold)
        num_choice = num_of_combination(possible_fsp_dtf)
        print(f"Applied {np.round(threshold, 1)} threshold. There are {num_choice} options of FSP DOF")
        threshold += 0.1

    # Generate combinations
    fsp = list(possible_fsp_dtf.keys())
    dtf_options = list(possible_fsp_dtf.values())
    combinations = [dict(zip(fsp, combo)) for combo in product(*dtf_options)]

    # 5. Initialize Optimization Variables
    total_edge_depth_optimize = [np.inf]
    idx_optimize = np.nan
    best_fsp_per_edge = {}

    # 6. Find the Best Combination
    for combo_idx, combination in enumerate(combinations):
        total_edge_depth = []
        edge_condition = True

        # Calculate flood depth for each edge pixel
        for j in range(gdfRS_edges_match.shape[0]):
            fsp = gdfRS_edges_match.loc[j, 'FspIds']
            dtf = gdfRS_edges_match.loc[j, 'Dtfs']
            dof = [combination[key] for key in fsp]

            flood_depth = get_depth(fsp, dtf, dof)
            if np.isnan(flood_depth):  # Stop if combination leads to unflooded edge
                edge_condition = False
                break
            else:
                total_edge_depth.append(flood_depth)

        if not edge_condition:
            continue

        # Update optimal combination if it minimizes total flood depth
        if sum(total_edge_depth) < sum(total_edge_depth_optimize):
            total_edge_depth_optimize = total_edge_depth
            idx_optimize = combo_idx

    # Update `best_case` with the optimal combination
    best_case = combinations[idx_optimize]
    print(f"Best case FSP combination: {best_case}")
    depth_weight = 1
    distance_weight = 1 
    elevation_weight = 1
    #epsilon = 1e-6 
    # 7. Rank FSPs Based on Flood Depth and Distance
    for j in range(gdfRS_edges_match.shape[0]):
        fsp_ids = gdfRS_edges_match.loc[j, 'FspIds']
        dtfs = gdfRS_edges_match.loc[j, 'Dtfs']
        edge_x, edge_y = gdfRS_edges_match.loc[j, 'X_coord'], gdfRS_edges_match.loc[j, 'Y_coord']
        fsp_x_coords = gdfRS_edges_match.loc[j, 'FSP_X_Coords']
        fsp_y_coords = gdfRS_edges_match.loc[j, 'FSP_Y_Coords']
        fpp_elevations = gdfRS_edges_match.loc[j, 'FPP_Elevation']
        fsp_elevations = gdfRS_edges_match.loc[j, 'FSP_Elevation']

        fsp_ranks = []
        for idx, fsp_id in enumerate(fsp_ids):
            dtf = dtfs[idx]
            fsp_x, fsp_y = fsp_x_coords[idx], fsp_y_coords[idx]
            fsp_elevation = fsp_elevations[idx]
            dof = best_case.get(fsp_id, None)
            if dof is None:
                continue

            flood_depth = max(0, dof - dtf)
            #print(f"{flood_depth:.2f} = {dof} - {dtf}")

            xdistance = abs(((edge_x - fsp_x) ** 2 + (edge_y - fsp_y) ** 2) ** 0.5)
            ydistance = fsp_elevation
            weighted_depth =  (flood_depth) * depth_weight
            weighted_distance = (1/ xdistance) * distance_weight 
            weighted_elevation = (1/ ydistance) * elevation_weight 

            fsp_ranks.append((fsp_id, weighted_depth, weighted_distance, weighted_elevation))
        
        # Rank by flood depth and distance
        flood_depths = [rank[1] for rank in fsp_ranks]
        distances = [rank[2] for rank in fsp_ranks]
        vertical_distance = [rank[3] for rank in fsp_ranks]

        flood_ranks = np.argsort(np.argsort(flood_depths)) + 1
        distance_ranks = np.argsort(np.argsort(distances)) + 1
        elevation_ranks = np.argsort(np.argsort(-np.array(vertical_distance))) + 1
        combined_ranks_list = []
        for k in range(len(fsp_ranks)):
            combined_value = (
                depth_weight*flood_ranks[k] +
                distance_weight*distance_ranks[k] +
                elevation_weight*elevation_ranks[k]
            )
            combined_ranks_list.append(combined_value)


        best_index = np.argmin(combined_ranks_list)
        best_fsp = fsp_ranks[best_index][0]
        best_fsp_per_edge[j] = best_fsp
        best_depth = fsp_ranks[best_index][1]
        best_distance = fsp_ranks[best_index][2]
        best_elevation_difference = fsp_ranks[best_index][3]
        best_combined_rank = combined_ranks_list[best_index]
        
        final_records.append({
            'Edge_Index': j,
            'Best_FSP': best_fsp,
            'Dof': best_case.get(best_fsp),
            'Flood_Depth': best_depth,
            'Distance': best_distance,
            'Elevation_Distance': best_elevation_difference,
            'Combined_Rank': best_combined_rank
        })   
    final_results_df = pd.DataFrame(final_records)

    print(f"{len(final_results_df)} edges pixels optimized")
    print(f"Minimum flood depth: {final_results_df['Flood_Depth'].min():.2f}")
    print(f"Median flood depth: {final_results_df['Flood_Depth'].median():.2f}")
    print(f"Mean flood depth: {final_results_df['Flood_Depth'].mean():.2f}")
    print(f"Max flood depth: {final_results_df['Flood_Depth'].max():.2f}")
    print("------------------------------------------------------------")

## Step 6.2: Go through all possible combinations
### Note: We use the set of true edges with the lowest NumofFSP to assign associated FSP Dofs and then iteratively move on to the set of true edge with higher NumofFSP. Fsps being assigned in each iteration doesn't change after each iteration. 

### Set the maximum number of possible combinations for each iteration. When the total number of possible combinations exceed this value, the FSP DOF dictionary is filtered with a 0.1 incremental threshold to remove Dofs options that are close to each other

1.  

In [ ]:
# #1. Setting Maximum Combinations and Initializing `best_case`
# max_combinations = 100000 # The maximum number of combinations to avoid excessive computations, if there are a lot of edge pixels and computation times are too long reduce this number
# best_case = {} #intilize dictionary to store the bestcase DOF for each FSP

# #2. Iterate through Different Limits of FSP Count per Edge  
# for i in range(gdfRS_edges_match_full.NumsFSP.min(),n + 1): 
#     print("NumofFSPs <= %s"%(i))
#     #3. Filter Data by FSP Count and Round `Dtfs` Values 
#     #limit the dataset to edges with Number of FSPs <=i and rest index
#     gdfRS_edges_match = gdfRS_edges_match_full[gdfRS_edges_match_full.NumsFSP <= i].reset_index(drop = True) # Limit to edge with <= n Fsps

#     #Round DTFs values to one decimal to reduce unique combinations 
#     gdfRS_edges_match.Dtfs = gdfRS_edges_match.Dtfs.apply(lambda x:np.round(x,1))
    
#     #4. Generate Possible DTF Values for Each FSP 
#     #Create a dictionary of FSP IDs and possible DTFs for each ID
#     possible_fsp_dtf = defaultdict(set)
#     for fspids, dtfs in zip(gdfRS_edges_match['FspIds'], gdfRS_edges_match['Dtfs']): 
#         for fspid, dtf in zip(fspids, dtfs):
#             possible_fsp_dtf[fspid].add(dtf)
    
#     #Convert each set to of possible DTF values to a list for easy combination generation
#     possible_fsp_dtf = {k: list(v) for k, v in possible_fsp_dtf.items()}
#     print('Total number of possible Fsp are %s'%(len(possible_fsp_dtf)))

#     #Update possible FSP choices with optimized values from the previous iteration
#     for key,value in best_case.items():
#         possible_fsp_dtf[key] = [value]

#     #Calculate the original number of combinations
#     num_choice = num_of_combination(possible_fsp_dtf)
#     print('There are %s options of FSP DOF '%(num_choice))

#     #Reduce options if the number of combinations exceeds the threshold 
#     threshold = 0.1
#     while num_choice > max_combinations:
#         possible_fsp_dtf = filter_close_values(possible_fsp_dtf, threshold)
#         num_choice = num_of_combination(possible_fsp_dtf)
#         print('Applied %s threshold. There are %s options of FSP DOF '%(np.round(threshold, 1), num_choice))
#         threshold += 0.1

#     #Generarte all possible combinations of FSP-DOF values
#     fsp = list(possible_fsp_dtf.keys())
#     dtf_options = list(possible_fsp_dtf.values())
#     combinations = [dict(zip(fsp, combo)) for combo in product(*dtf_options)]

#     #Initialize variables for optimize edge depth 
#     total_edge_depth_optimize = [np.inf] 
#     idx_optimize = np.nan #Store index of optimal combination 
    
#     # Find the best combination that minimize flood depth at edge
#     for i in range(len(combinations)): #Iterate through each combinations of FSPs' DOF
#         total_edge_depth = []# Accumulate flood depths for each combination
#         edge_condition = True # Track wheteher the combiation is valid
        
#         # Iterate through edge pixels to get the depth for each combination
#         for j in range( gdfRS_edges_match.shape[0]): 
#             fsp = gdfRS_edges_match.loc[j, 'FspIds']
#             dtf = gdfRS_edges_match.loc[j, 'Dtfs']  
#             dof = [combinations[i][key] for key in fsp]

#             # Calculate flood depth; stop if a non-flooded edge is ecnountered
#             flood_depth = get_depth(fsp,dtf,dof)
#             if np.isnan(flood_depth): # Stop calculating when the scenario produces a true edge that is not flooded
#                 edge_condition = False
#                 break
#             else:
#                 total_edge_depth.append(flood_depth)

#         # Skip to the next combination if this one is invalid         
#         if not edge_condition: # Go to the next combination
#             continue
#         else:
#             # Update optimal combination if current one has lower total flood depth 
#             if sum(total_edge_depth) < sum(total_edge_depth_optimize):
#                 total_edge_depth_optimize = total_edge_depth
#                 idx_optimize = i

#     # Update best-case FSP-DOF values with the current optimal combination          
#     best_case = combinations[idx_optimize]
#     print('%s FSP optimized' %(len(best_case)))
#     print('Total edge pixels: %s'%(gdfRS_edges_match.shape[0]))
#     print('Edge pixels that dont get flooded: %s'%(np.count_nonzero(np.isnan(total_edge_depth_optimize))))
#     print('Median flood depth at edge pixels: %s'%(np.nanmedian(total_edge_depth_optimize).round(2)))
#     print('Mean flood depth at edge pixels: %s'%(np.nanmean(total_edge_depth_optimize).round(2)))
#     print('Max flood depth at edge pixels: %s'%(np.nanmax(total_edge_depth_optimize).round(2)))
#     print('-------------------------------------------------------------------------------------------')


# Step 7: SNAP to FSP

In [ ]:
selected_fsp_ids = list(best_fsp_per_edge.values())

# Step 2: Create a new DataFrame from the original `fsp_gdf` with only selected FSPs
# Use the best-case FSPs' DOF values for the mapping
sol = fsp_gdf.set_index('FspId').loc[selected_fsp_ids].copy()
sol['Dof'] = sol.index.map(best_case)  # Map the optimized DOF values from `best_case`

# Step 3: Display the updated DataFrame
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#     display(sol)

# Step 4: Analyze and print summary statistics
# Print the counts of each `StrOrd` value
print(sol['StrOrd'].value_counts())

# Find the most common `StrOrd` value
highest_value = sol['StrOrd'].value_counts().index[0]
print(f"Most common StrOrd: {highest_value}")

In [ ]:
# # Create the FSP Geodataframe with Dof being the optimized Dof derived from either way above
# sol = fsp_gdf.set_index('FspId').loc[best_case.keys()]
# sol['Dof'] = sol.index.map(best_case)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#     display(sol)

# print(sol['StrOrd'].value_counts())

# highest_value = sol['StrOrd'].value_counts().index[0]
# print(highest_value)

### Note: In our study, we only kept assign FSPs on the mainstem. Skip this cell below if needed

In [ ]:
#sol = sol[(sol['StrOrd'] == highest_value)] # replace highest_value with any stream order number to select a specific stream order

gaugeFspDf = sol[['FspX', 'FspY', 'StrOrd', 'DsDist', 'SegId', 'FilledElev','Dof']].reset_index(drop = True)
gaugeFspDf.insert(0, 'lib_name', 'lib_fldsensing')
display(gaugeFspDf)

# Step 8: Interpolate FSP's DOF

Here we interpolate the DOF for all the FSPs between the gauge-FSPs using their DOF calculated from previous step. The interpolation uses stream orders and starts from low stream order (i.e., main streams) to high stream order (i.e., tributatried). Either horizontal or vertical (by defaut) interpolation can be used.

In [ ]:
# Find libs with snapped gauges. They are the actual libs to map
libs2Map = gaugeFspDf['lib_name'].drop_duplicates().tolist()
print(libs2Map)

# prepare the DF for storing interpolated FSP DOF
fspDof = pd.DataFrame(columns=['LibName','FspId','Dof'])

# prepare DFs for saving interpolated FSPs and their segment IDs
fspCols = fspInfoColumnNames + ['Dof']
segIdCols = ['SegId','LibName']
fsps = pd.DataFrame(columns=fspCols)
segIds =pd.DataFrame(columns=segIdCols)

# map each library
for libName in libs2Map:
    # interpolate DOF for the gauges
    # print('Interpolate FSP DOF using gauge DOF ...')
    # fspIdDof = InterpolateFspDofFromGauge(libFolder,libName,gaugeFspDf) # 'V' by default
    fspIdDof = InterpolateFspDofFromGauge(libFolder,libName,gaugeFspDf,weightingType='H') # horizontal interpolation
    fspIdDof['LibName'] = libName
    fspIdDofUse=fspIdDof.dropna()
    display(fspIdDofUse)

    # fspDof = fspDof.append(fspIdDof[['LibName','FspId','Dof']], ignore_index=True)
    fspDof = pd.concat([fspDof,fspIdDofUse[['LibName','FspId','Dof']]], ignore_index=True)
    fspDof['Dof'] = savgol_filter(fspDof['Dof'], 5001, 1)
    # Keep interpolated FSP DOF for saving later
    fspFile = os.path.join(libFolder, libName, fspInfoFileName)
    fspDf = pd.read_csv(fspFile) 
    fspDf = pd.merge(fspDf,fspDof,how='inner',on=['FspId'])
    # fsps = fsps.append(fspDf, ignore_index=True)
    fsps = pd.concat([fsps,fspDf], ignore_index=True)
    
    # Keep FSP segment IDs for saving later
    t =  pd.DataFrame(fspDf['SegId'].drop_duplicates().sort_values())
    t['LibName'] = libName
    # segIds = segIds.append(t, ignore_index=True)
    segIds = pd.concat([segIds,t], ignore_index=True)


# show interpolated FSPs with Dof
print(fspDof)

#
# save interpolated FSP DOF and their segments for checking. This block of code should be commented out if no-checking needed
#
# Save DOF and segment IDs to CSV files
FspDofFile = os.path.join(outputFolder, 'Interpolated_FSP_DOF.csv')
SegIdFile = os.path.join(outputFolder, 'Interpolated_SegIds.csv')

fsps.to_csv(FspDofFile, index=False)
segIds.to_csv(SegIdFile, index=False)

# Step 9: Map Flood Inundation Depth




The process of generating inundation depth map happens here.

In [ ]:
# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# Find libs needs mapping
libs2Map = fspDof['LibName'].drop_duplicates().tolist()
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        # cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='8GB',local_dir="D:\temp") # for office desktop (64G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    print(fspDof)
    # select the FSPs within the lib
    fspIdDof = fspDof[fspDof['LibName']==libName][['FspId','Dof']]
    print(fspIdDof)

    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        MosaicGtifs(outMapFolder,tileTifs,mosaicTifName,keepTifs=False)
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False

In [ ]:
fp = r"C:\Users\hobbe\Documents\Thesis\fldpln\Data\verdigris_10m_v8\maps\Testing9\lib_fldsensing_Testing9.tif"

raster = rasterio.open(fp)
fig, ax = plt.subplots(figsize=(25,25))
#image_hidden = ax.imshow(raster.read(1))
image_hidden = ax.imshow(raster.read(1, masked=True))

fig.colorbar(image_hidden, ax=ax)
rasterio.plot.show(raster, ax=ax)

### Turning the water surface elevator from the above FSPInfo folder into a raster

To display water surface elevation changes when changing various parameters

In [ ]:
# Load the CSV file
file_path = "C:/Users/hobbe/Documents/Thesis/fldpln/Data/verdigris_10m_v8/maps/Interpolated_FSP_DOF.csv"
fsp_data = pd.read_csv(file_path)

# Ensure StrOrd is numeric
fsp_data['StrOrd'] = pd.to_numeric(fsp_data['StrOrd'], errors='coerce')


# Calculate Water Surface Elevation (WSE) as FilledElev + Dof
fsp_data['WSE'] = fsp_data['FilledElev'] + fsp_data['Dof']
fsp_data['DsDist']=fsp_data['DsDist']
#print(fsp_data)

# Create a GeoDataFrame from the FSP coordinates
geometry = [Point(xy) for xy in zip(fsp_data['FspX'], fsp_data['FspY'])]
fsp_gdf = gpd.GeoDataFrame(fsp_data, geometry=geometry)

# Set the CRS to EPSG:26914 (UTM Zone 14N)
fsp_gdf.set_crs(epsg=26914, inplace=True)

# Create an empty raster with the desired resolution (e.g., 10 meters)
x_min, y_min, x_max, y_max = fsp_gdf.total_bounds
resolution = 10  # Example resolution (10 meters)
cols = int((x_max - x_min) / resolution) + 1  # +1 to include the edge
rows = int((y_max - y_min) / resolution) + 1  # +1 to include the edge
fspid_data = np.full((rows, cols), np.nan)

# Create an empty array for the raster data
raster_data = np.full((rows, cols), np.nan)
#print(fsp_gdf)

# Get the transform for the raster (defines the geo-referencing)
transform = from_origin(x_min, y_max, resolution, resolution)

# Fill in the raster array with WSE values at the FSP coordinates
for idx, row in fsp_gdf.iterrows():
    col = int((row.geometry.x - x_min) / resolution)
    row_idx = int((y_max - row.geometry.y) / resolution)
    
    # Ensure indices are within bounds
    if 0 <= row_idx < rows and 0 <= col < cols:
        raster_data[row_idx, col] = row['WSE']
        #fspid_data[row_idx, col] = row['FspId'] 
    else:
        print(f"Index out of bounds for coordinates: {row.geometry.x}, {row.geometry.y}")

# Define the raster's metadata
raster_meta = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': np.nan,
    'width': cols,
    'height': rows,
    'count': 1,
    'crs': fsp_gdf.crs,
    'transform': transform
}

# Save the raster as a GeoTIFF
print(os.getcwd())
output_tiff_path = 'WSE_rasterWithTributaries.tif'
with rasterio.open(output_tiff_path, 'w', **raster_meta) as dst:
    dst.write(raster_data.astype(np.float32), 1)

print(f"Raster saved as {output_tiff_path}")

fig, ax = plt.subplots(figsize=(10, 8))

tributaries = fsp_gdf[fsp_gdf['StrOrd'] > 1]
mainstem = fsp_gdf[fsp_gdf['StrOrd'] == 1]

print(fsp_gdf)
fsp_gdf.to_csv('WSEWithTrib.csv')

# Plot the mainstem and tributaries
mainstem.plot(ax=ax, marker='o', color='blue', markersize=5)
tributaries.plot(ax=ax, marker='x', color='red', markersize=5)

# Add a colorbar for the water surface elevation (WSE) based on 'FilledElev'
mainstem.plot(column='WSE', cmap='Reds', ax=ax, legend=True)

# Add titles and labels
ax.set_title('Mainstem FSP Water Surface Elevation with Tributaries')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')

plt.legend()
plt.show()

In [ ]:
dstData = fsp_gdf.query('StrOrd ==1')['DsDist']
print(dstData)
wseData = fsp_gdf.query('StrOrd ==1')['WSE']
print(wseData)
filledElevData = fsp_gdf.query('StrOrd ==1')['FilledElev']
f = plt.figure()
plt.title('Mainstem FSP Water Surface Elevation with Tributaries')
plt.xlabel('Downstream Distance')
plt.ylabel('Y Coordinate')
f.set_figwidth(10)
f.set_figheight(5)
plt.plot(dstData,wseData)